In [18]:
!pip install -q yfinance
!pip install -q matplotlib
!pip install -q pandas_datareader
!pip install -q lightgbm
!pip install scikit-learn
!pip install lightgbm --upgrade
!pip install graphviz




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\theaw\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import numpy as np
import datetime
import lightgbm
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import graphviz
from sklearn.preprocessing import LabelEncoder





# Economy Sample Stock

In [2]:
def stock_info(macro_indicators):
    ticker_data_info = pd.DataFrame()
    tick = yf.download(ticker, start = start_date_with_shift, end = end_date)
    ticker_data_info['average_stock_price'] = tick['Close'].resample(period_length).mean()
    ticker_data_info['average_stock_price_change'] = ticker_data_info['average_stock_price'] - ticker_data_info['average_stock_price'].shift(1)
    ticker_data_info['lag_average_stock_price_change'] = ticker_data_info['average_stock_price_change'].shift(1)
    ticker_data_info.reset_index(inplace=True)
    macro_indicators = pd.merge(macro_indicators, ticker_data_info[['Date', 'average_stock_price', 'average_stock_price_change', 'lag_average_stock_price_change']], 
                                left_on = 'DATE',
                                right_on = 'Date',
                                how = 'left')
    macro_indicators.drop(columns=['Date'], inplace=True)
    
    return macro_indicators


# Money Supply

In [3]:
def fetch_money_supply():
    # Define the list of money supply series to fetch
    money_supply_series = ['M1SL', 'M2SL']

    # Fetch money supply data using pandas_datareader
    money_supply_data = pdr.get_data_fred(money_supply_series, start_date_with_shift, end_date)

    # Calculate the YoY percentage change
    money_supply_data['M1SL_YoY'] = money_supply_data['M1SL'].pct_change(12) * 100
    money_supply_data['M2SL_YoY'] = money_supply_data['M2SL'].pct_change(12) * 100


    return money_supply_data


In [4]:
# Retrieves all MS between start and end date and returns MS differences in each period bin
def money_supply_info(macro_indicators):
    money_supply_data = fetch_money_supply()
    
    money_supply_data_average = money_supply_data.resample(period_length).mean()

    money_supply_data_average['M1-M2'] = money_supply_data_average['M1SL_YoY'] - money_supply_data_average['M2SL_YoY']

    macro_indicators = pd.merge(macro_indicators, money_supply_data_average['M1-M2'], on='DATE', how='left')
    return macro_indicators

# CPI Inflation

In [5]:
def fetch_cpi_annual_change():
    try:
        # Fetch U.S. Consumer Price Index (CPI) data from FRED
        cpi_data = pdr.get_data_fred('CPIAUCSL', start = start_date_with_shift, end = end_date)

        if not cpi_data.empty:
            # Calculate annual percentage change
            cpi_data['Annual_Change'] = cpi_data['CPIAUCSL'].pct_change(12) * 100

            return cpi_data[['Annual_Change']]
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error


In [6]:
def cpi_info(macro_indicators):
    cpi_data_info = pd.DataFrame()
    cpi_data = fetch_cpi_annual_change()
    cpi_data_info['average_cpi_change'] = cpi_data.resample(period_length).mean()
    cpi_data_info['single_average_cpi_change'] = cpi_data_info['average_cpi_change'] - cpi_data_info['average_cpi_change'].shift(1)
    cpi_data_info['lag_single_average_cpi_change'] = cpi_data_info['single_average_cpi_change'].shift(1)

    macro_indicators = pd.merge(macro_indicators, cpi_data_info[['average_cpi_change', 'single_average_cpi_change', 'lag_single_average_cpi_change']], on = 'DATE', how = 'left')
    
    return(macro_indicators)

# Unemployment

In [7]:
def fetch_unemployment_data():
    try:
        # Define the ticker symbol for the U.S. unemployment rate (UNRATE)
        unemployment_ticker = "UNRATE"

        # Fetch unemployment data from FRED
        unemployment_data = pdr.get_data_fred(unemployment_ticker, start = start_date_with_shift, end = end_date)

        return unemployment_data
    except Exception as e:
        return None  # Return None if there is an error

In [8]:
def unemployment_info(macro_indicators):
    unemployment_info = pd.DataFrame()
    unemployment_data = fetch_unemployment_data()
    unemployment_info['average_unemployment'] = unemployment_data.resample(period_length).mean()
    unemployment_info['average_unemployment_change'] = unemployment_info['average_unemployment'] - unemployment_info['average_unemployment'].shift(1)
    unemployment_info['lag_average_unemployment_change'] = unemployment_info['average_unemployment_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, unemployment_info[['average_unemployment', 'average_unemployment_change', 'lag_average_unemployment_change']], on = 'DATE', how = 'left')
    return macro_indicators

# Interest Rates

In [9]:
def fetch_fed_interest_rate():
    try:
        # Fetch Federal Reserve interest rate data (FEDFUNDS) from FRED
        fed_interest_rate = pdr.get_data_fred('FEDFUNDS', start = start_date_with_shift, end=end_date)

        if not fed_interest_rate.empty:
            return fed_interest_rate
        else:
            return None  # Return None if there is no data
    except Exception as e:
        return None  # Return None if there is an error



In [10]:
def fed_interest_rate_info(macro_indicators):
    fed_interest_rate_info = pd.DataFrame()
    fed_interest_rate = fetch_fed_interest_rate()
    fed_interest_rate_info['average_interest_rate'] = fed_interest_rate.resample(period_length).mean()
    fed_interest_rate_info['single_rate_period_change'] = fed_interest_rate_info['average_interest_rate'] - fed_interest_rate_info['average_interest_rate'].shift(1)
    fed_interest_rate_info['lag_single_rate_change'] = fed_interest_rate_info['single_rate_period_change'].shift(1)
    macro_indicators = pd.merge(macro_indicators, fed_interest_rate_info[['average_interest_rate','single_rate_period_change', 'lag_single_rate_change']],on = 'DATE', how = 'left')
    return(macro_indicators)



# CPI, Inflation, unemployment, interest rates


In [11]:
# New Function that takes the df(macro_indicators), function name, type of indicator
# Not currently used in my code yet but more efficent coding :)
def calculate_indicator_info(macro_indicators, fetch_function, column_prefix):
    indicator_info = pd.DataFrame()
    indicator_data = fetch_function()
    average_column_name = f'average_{column_prefix}'
    single_change_column_name = f'single_{column_prefix}_change'
    lag_single_change_column_name = f'lag_{single_change_column_name}'

    indicator_info[average_column_name] = indicator_data.resample(period_length).mean()
    indicator_info[single_change_column_name] = indicator_info[average_column_name] - indicator_info[average_column_name].shift(1)
    indicator_info[lag_single_change_column_name] = indicator_info[single_change_column_name].shift(1)

    macro_indicators = pd.merge(
        macro_indicators,
        indicator_info[[average_column_name, single_change_column_name, lag_single_change_column_name]],
        on='DATE',
        how='left'
    )

    return macro_indicators

In [12]:
def apply_indicators_info(macro_indicators, indicator_functions):
    for indicator_function in indicator_functions:
        macro_indicators = indicator_function(macro_indicators)
    return macro_indicators

# 

# MAIN

In [14]:
# Main
period_length = '1M'
ticker = "SPY"
end_date = datetime.datetime(2022, 1, 1)

# Not sure why i can only do 36 months ahead
start_date = end_date - relativedelta(months = 36)
start_date_with_shift = start_date - relativedelta(months = 15)

def get_data(end_date):

    date_range = pd.date_range(start=start_date, end=end_date, freq = period_length)
    macro_indicators = pd.DataFrame({'DATE': date_range})

    stock_info(macro_indicators)

    indicator_functions = [stock_info, money_supply_info, fed_interest_rate_info, cpi_info, unemployment_info]
    macro_indicators = apply_indicators_info(macro_indicators, indicator_functions)
    return macro_indicators





In [15]:
end_date = datetime.datetime(2022, 1, 1)
macro_indicators = get_data(end_date)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


# BACKTESTING

### SELL, HOLD, BUY


In [16]:
# Determine SELL, HOLD, or BUY based on future change
def buy_or_sell(training_macro_indicators):
    training_macro_indicators['future_change'] = training_macro_indicators['average_stock_price'].shift(-1) - training_macro_indicators['average_stock_price']

    conditions = [
        (training_macro_indicators['future_change'] >= 5),
        (training_macro_indicators['future_change'] < 5) & (training_macro_indicators['future_change'] > -5),
        (training_macro_indicators['future_change'] <= -5)
    ]

    choices = ['BUY', 'HOLD', 'SELL']

    training_macro_indicators['buy_or_sell'] = np.select(conditions, choices, default='HOLD')

    return training_macro_indicators

In [17]:
train_macro_indicators = macro_indicators

In [18]:
full_train_macro_indicators = buy_or_sell(train_macro_indicators)
full_train_macro_indicators.tail()

,DATE,average_stock_price,average_stock_price_change,lag_average_stock_price_change,M1-M2,average_interest_rate,single_rate_period_change,lag_single_rate_change,average_cpi_change,single_average_cpi_change,lag_single_average_cpi_change,average_unemployment,average_unemployment_change,lag_average_unemployment_change,future_change,buy_or_sell
31,2021-08-31,444.520000,9.425718,12.109738,3.279482,0.09,-0.01,0.02,5.188292,-0.033214,-0.060105,5.2,-0.2,-0.5,-0.966189,HOLD
32,2021-09-30,443.553811,-0.966189,9.425718,2.929079,0.08,-0.01,-0.01,5.383630,0.195338,-0.033214,4.8,-0.4,-0.2,1.171426,HOLD
33,2021-10-31,444.725237,1.171426,-0.966189,2.683967,0.08,0.00,-0.01,6.237754,0.854124,0.195338,4.5,-0.3,-0.4,21.030478,BUY
34,2021-11-30,465.755716,21.030478,1.171426,2.509540,0.08,0.00,0.00,6.862388,0.624634,0.854124,4.2,-0.3,-0.3,0.649286,HOLD
35,2021-12-31,466.405002,0.649286,21.030478,2.224443,0.08,0.00,0.00,7.194459,0.332071,0.624634,3.9,-0.3,-0.3,NaN,HOLD


### LGBM MODEL

In [20]:
var_columns = [c for c in full_train_macro_indicators if c not in['future_change','DATE','buy_or_sell']]

X = full_train_macro_indicators.loc[:, var_columns]
y = full_train_macro_indicators.loc[:, 'buy_or_sell']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

In [21]:
train_data = lightgbm.Dataset(X_train, label = y_train)
valid_data = lightgbm.Dataset(X_valid, label = y_valid)

In [30]:

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_valid_encoded = label_encoder.transform(y_valid)

params = {
    'objective': 'multiclass',
    'num_class': 3,
    'metric': 'multi_logloss',
    'min_data_in_bin': 5,
    'min_data_in_leaf': 10
}

train_data_encoded = lightgbm.Dataset(X_train, label=y_train_encoded)
valid_data_encoded = lightgbm.Dataset(X_valid, label=y_valid_encoded)

model = lightgbm.train(
    params,
    train_data_encoded,
    valid_sets=[valid_data_encoded],
    num_boost_round=5
)

predictions = model.predict(X_valid)
predicted_labels = predictions.argmax(axis=1)

try:
    # Convert predicted labels to match the format of true labels
    predicted_labels = ['BUY' if label == 0 else 'HOLD' for label in predicted_labels]

    # Evaluate the model
    accuracy = accuracy_score(y_valid, predicted_labels)
    print(f'Accuracy: {accuracy}')
except Exception as e:
    print(f'Error: {e}')

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002538 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 89
[LightGBM] [Info] Number of data points in the train set: 28, number of used features: 13
[LightGBM] [Info] Start training from score -0.559616
[LightGBM] [Info] Start training from score -0.934309
[LightGBM] [Info] Start training from score -3.332205
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

In [31]:

# Assuming model is your trained LightGBM model
graph = lightgbm.create_tree_digraph(model, tree_index=0, format='png', graph_attr={'size': '20,8'})
graph.render('tree')  # This will save a PNG file with the tree visualization
graph.view('tree')

ExecutableNotFound: failed to execute WindowsPath('dot'), make sure the Graphviz executables are on your systems' PATH